In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [3]:
train=pd.read_csv("../input/transaction-fruad/Train_without_Data_balancing.csv")
train=train.drop(train.columns[0],axis=1)
Train_target_df=train.loc[:,"isFraud"]
train.drop("isFraud",axis=1,inplace=True)
X_train, X_test, y_train, y_test = train_test_split(train, Train_target_df, test_size=0.20, random_state=0)

In [4]:
import tensorflow as tf
from sklearn.metrics import roc_auc_score, f1_score

# Set the input size and number of classes
input_size = X_train.shape[1]
num_classes = 1

# Create the model
model = tf.keras.Sequential()

# Add the 3 inner layers with ReLU activation function
model.add(tf.keras.layers.Dense(units=64, activation='relu', input_shape=(input_size,)))
model.add(tf.keras.layers.Dense(units=32, activation='relu'))
model.add(tf.keras.layers.Dense(units=16, activation='relu'))

# Add the output layer with sigmoid activation function
model.add(tf.keras.layers.Dense(units=num_classes, activation='sigmoid'))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision()])

2022-12-14 07:46:11.962973: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
# Train the model
model.fit(X_train, y_train, epochs=10)

2022-12-14 07:46:13.119853: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
11073/11073 [==============================] - 41s 4ms/step - loss: 0.5241 - accuracy: 0.9555 - auc: 0.6011 - precision: 0.1516
Epoch 2/10
11073/11073 [==============================] - 39s 4ms/step - loss: 0.1410 - accuracy: 0.9653 - auc: 0.7260 - precision: 0.5599
Epoch 3/10
11073/11073 [==============================] - 38s 3ms/step - loss: 0.1331 - accuracy: 0.9666 - auc: 0.7576 - precision: 0.7446
Epoch 4/10
11073/11073 [==============================] - 41s 4ms/step - loss: 0.1254 - accuracy: 0.9684 - auc: 0.7814 - precision: 0.7791
Epoch 5/10
11073/11073 [==============================] - 41s 4ms/step - loss: 0.1221 - accuracy: 0.9693 - auc: 0.7943 - precision: 0.7987
Epoch 6/10
11073/11073 [==============================] - 41s 4ms/step - loss: 0.1189 - accuracy: 0.9697 - auc: 0.8057 - precision: 0.7878
Epoch 7/10
11073/11073 [==============================] - 39s 4ms/step - loss: 0.1165 - accuracy: 0.9702 - auc: 0.8135 - precision: 0.7992
Epoch 8/10
11073/11073 [===

In [6]:
# Make predictions on the test set
y_pred = model.predict(X_test)

In [7]:
# Evaluate the model
loss, accuracy, auc, precision = model.evaluate(X_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)
print('AUC:', auc)
print('Precision:', precision)

2769/2769 [==============================] - 7s 2ms/step - loss: 0.1142 - accuracy: 0.9696 - auc: 0.8341 - precision: 0.8397
Loss: 0.11423464119434357
Accuracy: 0.9696323275566101
AUC: 0.8341042399406433
Precision: 0.8397058844566345


In [8]:
test=pd.read_csv("../input/transaction-fruad/Test_without_Data_balancing.csv")
test=test.drop(test.columns[0],axis=1)
predictions_test = model.predict(test)
Test_df_predictions=pd.DataFrame(data=predictions_test,columns=["isFraud"])
Test_df_predictions[Test_df_predictions['isFraud']>0.7]=1
Test_df_predictions[Test_df_predictions['isFraud']<=0.7]=0
Test_df_predictions.reset_index(inplace=True)
Test_df_predictions.rename(columns={"index":"Id"},inplace=True)

In [9]:
Test_df_predictions.head()

,Id,isFraud
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0


In [10]:
Test_df_predictions.shape

(147635, 2)

In [11]:

Test_df_predictions.to_csv("./output_predictions.csv",index=False)